# Capstone Project - Clustering Toronto

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import requests

### Import the csv and read it with pandas

In [2]:
toronto_df = pd.read_csv("toronto_geo_postal.csv")

In [3]:
toronto_df = toronto_df.drop(["Unnamed: 0"], axis=1)

In [4]:
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Get the coordinates of Toronto

In [5]:
adress = "Bata shoe museum, Toronto, Canada"

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(adress)
latitude = location.latitude
longitude = location.longitude

print(location)
print(latitude)
print(longitude)

Bata Shoe Museum, Bloor Street West, The Annex, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5S 1W7, Canada
43.66725785
-79.40014314836553


### Create a map of Toronto with Folium

#### Let's create a map object of Toronto

In [6]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles="Stamen Terrain")

#### Now, add a marker for each neighborhood of Toronto

Since there is row with more than one neighborhood for one borough, the borough are in uppercase for a better reading.

In [7]:
for postal_code, latitude, longitude, borough, neighborhood in zip(toronto_df["Postal Code"], toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}, {}'.format(postal_code, neighborhood, borough.upper())
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(toronto_map)  
    

In [8]:
toronto_map

In order to make it easier, we keep our focus on the Borough with only Toronto in the name. So we create a new datframe called toronto_borough_df

In [9]:
toronto_borough_df = toronto_df[toronto_df["Borough"].str.contains("Toronto")]
toronto_borough_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [10]:
toronto_borough_df.shape

(39, 5)

### Getting the new coordinate of the downtown of toronto

In [11]:
adress_2 = "Downtown Toronto, Toronto, Canada"

geolocator = Nominatim(user_agent="Toronto_explorer")
location_2 = geolocator.geocode(adress_2)
latitude_2 = location.latitude
longitude_2 = location.longitude

print(location)
print(latitude)
print(longitude)

Bata Shoe Museum, Bloor Street West, The Annex, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5S 1W7, Canada
43.6288408
-79.52099940000002


### Creating a new map of Toronto with the Downtown only

#### We attribute a new name to the map

In [12]:
toronto_borough_map = folium.Map(location=[latitude_2, longitude_2], zoom_start=11, tiles="Stamen Terrain")

In [13]:
for postal_code, latitude, longitude, borough, neighborhood in zip(toronto_borough_df["Postal Code"], toronto_borough_df['Latitude'], toronto_borough_df['Longitude'], toronto_borough_df['Borough'], toronto_borough_df['Neighborhood']):
    label = '{}, {}, {}'.format(postal_code, neighborhood, borough.upper())
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(toronto_borough_map)  

In [14]:
toronto_borough_map

### Using Foursquare API to explore the each neighborhood

#### I remove the cells with my CLIENT ID and SECRET. No offense..

Writting the code for one location: neighborhood of The Beaches

In [16]:
neighborhood = toronto_borough_df.loc[19, "Neighborhood"]

In [17]:
neighborhood

'The Beaches'

Defining the variable name, latitude and longitude for the selected neighborhood

In [18]:
neighborhood_name = toronto_borough_df.loc[19, "Neighborhood"]
neighborhood_latitude = toronto_borough_df.loc[19, "Latitude"]
neighborhood_longitude = toronto_borough_df.loc[19, "Longitude"]

print(neighborhood_name)
print(neighborhood_latitude)
print(neighborhood_longitude)

The Beaches
43.67635739999999
-79.2930312


#### Making the Foursquare API request

Now that we have the coordinates and the name, we create an url to get the top 50 venues for the The Beaches Neihghborhood in a 500 meters radius

In [19]:
RADIUS = 500
LIMIT = 50

url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(CLIENT_ID, 
                                                                                                                           CLIENT_SECRET, 
                                                                                                                           neighborhood_latitude, 
                                                                                                                           neighborhood_longitude, 
                                                                                                                           VERSION, 
                                                                                                                           RADIUS, 
                                                                                                                           LIMIT)

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f060c58521bc909eb0798e6'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


Now we have a dataframe containing the venues for The Beaches neighborhood

In [23]:
nearby_venues.shape

(4, 4)

There is only for rows and for columns for this nearby_venues dataframe.

We apply this code to all the neighborhood with the function getNearbyVenues

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The results are in the dataframe toronto_venues

In [26]:
toronto_venues = getNearbyVenues(names=toronto_borough_df['Neighborhood'],
                                   latitudes=toronto_borough_df['Latitude'],
                                   longitudes=toronto_borough_df['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [27]:
print(type(toronto_venues))
print(toronto_venues.shape)

<class 'pandas.core.frame.DataFrame'>
(1194, 7)


In [28]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


We can check the number of venues per Neighorhood

In [29]:
toronto_venues.groupby("Neighborhood").count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton, Parkdale Village, Exhibition Place",26,26,26,26,26,26
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,50,50,50,50,50,50


In order to apply a Kmeans algorithm, we need to convert the string type of the Venue Category to a numeric one. We use the get_dummies fonction.

In [30]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

for index, name in enumerate(toronto_onehot.columns):
    print(index, name)
# neighborhood column is at the index 149


0 Airport
1 Airport Food Court
2 Airport Gate
3 Airport Lounge
4 Airport Service
5 Airport Terminal
6 American Restaurant
7 Antique Shop
8 Aquarium
9 Art Gallery
10 Art Museum
11 Arts & Crafts Store
12 Asian Restaurant
13 Auto Workshop
14 BBQ Joint
15 Baby Store
16 Bagel Shop
17 Bakery
18 Bank
19 Bar
20 Baseball Stadium
21 Basketball Stadium
22 Beach
23 Beer Bar
24 Beer Store
25 Belgian Restaurant
26 Bistro
27 Board Shop
28 Boat or Ferry
29 Bookstore
30 Boutique
31 Brazilian Restaurant
32 Breakfast Spot
33 Brewery
34 Bubble Tea Shop
35 Burger Joint
36 Burrito Place
37 Bus Line
38 Butcher
39 Café
40 Cajun / Creole Restaurant
41 Candy Store
42 Caribbean Restaurant
43 Cheese Shop
44 Chinese Restaurant
45 Chocolate Shop
46 Church
47 Climbing Gym
48 Clothing Store
49 Cocktail Bar
50 Coffee Shop
51 College Arts Building
52 College Auditorium
53 College Gym
54 College Rec Center
55 Colombian Restaurant
56 Comfort Food Restaurant
57 Comic Shop
58 Concert Hall
59 Convenience Store
60 Cosmetics 

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0625
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0200


In [32]:
toronto_grouped.shape

(39, 212)

Now we print the top 5 venues for each Neighborhood

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1  Cocktail Bar  0.06
2          Café  0.04
3   Cheese Shop  0.04
4      Beer Bar  0.04


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.12
1                 Bakery  0.08
2         Breakfast Spot  0.08
3              Nightclub  0.08
4  Performing Arts Venue  0.08


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
              venue  freq
0       Yoga Studio  0.06
1     Garden Center  0.06
2        Comic Shop  0.06
3              Park  0.06
4  Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3           Airport  0.07
4             Plane  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.1

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now, we arrange all of the results into a Dataframe

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Bakery,Café,Farmers Market,Cheese Shop,Beer Bar,Greek Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Performing Arts Venue,Bakery,Breakfast Spot,Coffee Shop,Pet Store,Burrito Place,Stadium,Bar
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Skate Park,Brewery,Smoke Shop,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Harbor / Marina,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Spa,Donut Shop,Hotel,Ice Cream Shop,Indian Restaurant


The Dataframe contain the neighborhood and the 10 most common venues. We are ready to cluster those neighborhood accordingly to those 10 venues

### Clustering

We define a number of cluster equal to 4. We also drop the Neighborhood column wich is in a string format and also because it is a label. We only want the clustering to be made with the venues.

In [36]:
from sklearn.cluster import KMeans

kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:15] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0])

In [37]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_borough_df


Now we merge toronto_merge and neighborhoods_venues_sorted together to combine in the same dataframe the venues, the coordinates, the borough and the neighborhood and the postal code.

In [38]:
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [39]:
toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Breakfast Spot,Restaurant,Café,Pub,Theater,Yoga Studio,Dessert Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park,Music Venue
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Café,Tea Room,Theater,Cosmetics Shop,Fast Food Restaurant,Clothing Store,Ramen Restaurant,Bookstore,Sandwich Place
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Restaurant,Coffee Shop,Cosmetics Shop,Farmers Market,Gastropub,Creperie,Japanese Restaurant,Hotel,Seafood Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Yoga Studio,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


We print the map of the clustering

In [40]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Thank you